In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import joblib
import torch.nn as nn
import numpy as np
import cv2
import torch.nn.functional as F
import time
import cnn_models
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
%matplotlib inline

Loading label binarizer...


In [7]:
# load label binarizer
pred_text = ""
lb = joblib.load('../outputs/lb.pkl')
model = cnn_models.CustomCNN()
model.load_state_dict(torch.load('../outputs/model.pth', map_location='cpu'))
print(model)
print('Model loaded')

def hand_area(img):
    hand = img[100:324, 100:324]
    hand = cv2.resize(hand, (224,224))
    return hand


cap = cv2.VideoCapture(0)
if (cap.isOpened() == False):
    print('Error while trying to open camera. Plese check again...')
# get the frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
# define codec and create VideoWriter object
out = cv2.VideoWriter('../outputs/asl.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width,frame_height))

# read until end of video
while(cap.isOpened()):
    # capture each frame of the video
    ret, frame = cap.read()
    print('frame')
    print(frame)
    # get the hand area on the video capture screen
    cv2.rectangle(frame, (100, 100), (324, 324), (20, 34, 255), 2)
    hand = hand_area(frame)
    image = hand
    
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    image = torch.tensor(image, dtype=torch.float)
    image = image.unsqueeze(0)
    
    outputs = model(image)
    _, preds = torch.max(outputs.data, 1)
    
    cv2.putText(frame, lb.classes_[preds], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    current_letter = ''
    current_letter+=(lb.classes_[preds])
    print(current_letter)
    
    pred_text = pred_text +" "+ lb.classes_[preds]
#     print(lb.classes_[preds])
    cv2.imshow('image', frame)
    out.write(frame)
    # press `q` to exit
    if cv2.waitKey(27) & 0xFF == ord('q'):
        break
# release VideoCapture()
cap.release()
# close all frames and video windows
cv2.destroyAllWindows()

print("\n\n\nPredicted Output:-")
print(pred_text)


CustomCNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=29, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Model loaded
frame
None


TypeError: 'NoneType' object is not subscriptable

In [16]:
t = str(pred_text)
t = t.replace("nothing", "")
t = t.replace("space", "")
t = t.replace("del", "")
t = t.replace(" ", "")
t

'KLVVWWWWWWWWWWWWWWKKWWWWWWWWWWWWWWWWWWWWWWWWWFWWWEEEEEEEEEEALKKKKKKKDKEECCCCCCAEAEEEEEEEEAAAAMAGAGGGGMNAAMANNMAAIIIKKKKKKKKKKKKKKKKDBDDDDDDDDDCLFFCCCDDDDDDDDCCCCDBDDDDDDDDDFFDDDFFFWFWWWWFEEEEEEEEEALAAAMNNBBBBNBBBBNNNNBBBBBBBBBBBCVRUXXXVREERRX'

In [17]:
import torch
import torch.nn as nn
from preparing_data import all_categories
from create_network import *
from training import *

output_name = ""
out_name = []
 #Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)
        out_name.append(output_name)


# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        sample(category, start_letter)
print("Processing........")
for i in all_categories:
    samples(i, t)

import numpy as np
print("Predicted words from Video:- \n")
# function to get unique values
def unique(list1):
    x = np.array(list1)
    print(np.unique(x))
    
unique(out_name)

Processing........
Predicted words from Video:- 

['Aanian' 'Aenian' 'Aerere' 'Aererin' 'Aererini' 'Aereris' 'Aerinin'
 'Banian' 'Benian' 'Bereatin' 'Berele' 'Berere' 'Bereres' 'Bererin'
 'Bererini' 'Berinin' 'Carelin' 'Celinin' 'Cereatin' 'Cereatini' 'Cerere'
 'Cererin' 'Cerinin' 'Denian' 'Derelin' 'Derere' 'Dererin' 'Dererini'
 'Dereris' 'Dorere' 'Eanerin' 'Eanialin' 'Eanian' 'Eenian' 'Eenier'
 'Eerelin' 'Eererali' 'Eerere' 'Eererie' 'Eererin' 'Eererine' 'Eereris'
 'Einerin' 'Faniani' 'Fereatin' 'Ferere' 'Fererin' 'Fererini' 'Ferinin'
 'Foniali' 'Genian' 'Gerere' 'Gererin' 'Gerinin' 'Ienialin' 'Ieralin'
 'Iereatin' 'Kaneatin' 'Kanian' 'Kelinin' 'Kenian' 'Kenier' 'Kereatin'
 'Kereatis' 'Kerere' 'Kererin' 'Kererine' 'Kererini' 'Kerinin' 'Lanialin'
 'Lererin' 'Lererini' 'Merere' 'Mererin' 'Nanian' 'Nelerin' 'Nerere'
 'Nererin' 'Nereris' 'Rererin' 'Rerinin' 'Uanianin' 'Venian' 'Verealin'
 'Vererin' 'Wanian' 'Wenialin' 'Wenian' 'Werelin' 'Werere' 'Wereria'
 'Wererin' 'Wererini' 'Wererinis